In [34]:
import matplotlib.pyplot as plt
import sklearn.datasets as skdata
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

%matplotlib inline

numeros = skdata.load_digits()
target = numeros['target']
imagenes = numeros['images']
n_imagenes = len(target)

data = imagenes.reshape((n_imagenes, -1))
scaler = StandardScaler()
x_train, x_test, y_train, y_test = train_test_split(data, target, train_size=0.7)
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

cov = np.cov(x_train.T)
valores, vectores = np.linalg.eig(cov)
valores = np.real(valores)
vectores = np.real(vectores)
ii = np.argsort(-valores)
valores = valores[ii]
vectores = vectores[:,ii]

#Definimos una prob por cada componente de acuerdo a los mu y sigmas
def prob(x, vector, mu, sigma):
    Ps=1
    for i in x:
        comp = np.dot(i,vector)
        Ps=Ps*1/(sigma*np.sqrt(2*np.pi))*np.exp(-1/2*((comp-mu)/sigma)**2) 
    return Ps

sigmas = np.zeros((10,20))
mus = np.zeros((10,20))
for i in range(10):
    x_t = x_train[y_train == i]
    for j in range(20):
        mus[i,j] = np.mean(x_t @ vectores[:,j])
        Pm = 0
        for sig in np.linspace(0.001,20,1000):
            Pp = prob(x_t, vectores[:,j], mus[i,j], sig)
            if Pp > Pm:
                Pm = Pp
                sigmas[i,j] = sig
                
def probdig(x, dig):
    Probl = np.zeros(20)
    for i in range(20):
        comp = np.dot(x, vectores[:,i])
        Probl[i] = 1/(sigmas[dig,i]*np.sqrt(2*np.pi))*np.exp(-1/2*((comp-mus[dig,i])/sigmas[dig,i])**2)
    P = np.prod(Probl)
    return P

def predict(x):
    Probs = np.zeros(10)
    for i in range(10):
        Probs[i] = probdig(x, i)
    return np.argmax(Probs)

In [37]:
N = 540
Predict = np.zeros(N)
Etiqueta = np.zeros(N)
for muestra in range(N):
    xm = x_test[muestra]
    Predict[muestra] = predict(xm)
    Etiqueta[muestra] = y_test[muestra]
print(sum(Etiqueta == Predict)/N)

#for i in range(N):
#    print(Etiqueta[i], Predict[i])

0.8722222222222222
